In [1]:
import polaris
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from qsarcons.lazy import LazyML
from qsarcons.consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor

/storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1. Load data

In [3]:
# Load the benchmark from the Hub
benchmark = polaris.load_benchmark("polaris/adme-fang-solu-1")

# Get the train and test data-loaders
data_train, data_test = benchmark.get_train_test_split()
data_train, data_test = data_train.as_dataframe(), data_test.as_dataframe()

smi_train, prop_train = data_train["smiles"].to_list(), data_train["LOG_SOLUBILITY"].to_list()

data_train, data_val = train_test_split(data_train, test_size=0.2, random_state=42)

[2025-10-06 07:14:45] INFO     The version of Polaris that was used to create the artifact (0.0.0)  ]8;id=666880;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=234449;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.13.0).                                                                           

                      INFO     To verify the checksum, we need to recompute it. This can be slow    ]8;id=292307;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=780069;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py#67\67]8;;\
                               for large datasets.                                                                 

[07:14:45]  Success: Fetching dataset                                                                 ]8;id=192660;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=185980;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

                      INFO     The version of Polaris that was used to create the artifact (0.0.0)  ]8;id=957055;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=59990;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.13.0).                                                                           

                      WARNING  No checksum to verify against. Specify either the md5sum parameter   ]8;id=898014;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=316851;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py#60\60]8;;\
                               or store the checksum in the dataset.md5sum attribute.                              

            Success: Fetching benchmark                                                               ]8;id=803337;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=922660;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

### 2. Build multiple 2D models

In [4]:
data_test["LogS"] = [0 for i in data_test.index]

# lazy_ml = LazyML(task="regression", output_folder="logs_bench", verbose=True)
# lazy_ml.run(data_train, data_val, data_test)

Total models to build: 152


100%|█████████████████████████████████████████████████████████████████████████████████| 152/152 [20:45<00:00,  8.19s/it]

All models completed.


### 3. Build model consensus

In [5]:
metric = "auto"
cons_size = "auto"

In [7]:
cons_methods = [
    ("Best", SystematicSearchRegressor(cons_size=1, metric=metric)),         
    ("Random", RandomSearchRegressor(cons_size=cons_size, n_iter=1000, metric=metric)),       
    ("Systematic", SystematicSearchRegressor(cons_size=cons_size, metric=metric)),
    ("Genetic", GeneticSearchRegressor(cons_size=cons_size, n_iter=50, pop_size=50, mut_prob=0.2, metric=metric))
]

In [10]:
# load model predictions
df_val = pd.read_csv("logs_bench/val.csv")
df_test = pd.read_csv("logs_bench/test.csv")

# skip first two columns (smiles and true property value)
x_val, true_val = df_val.iloc[:, 2:], df_val.iloc[:, 1]
x_test = df_test.iloc[:, 2:]

In [11]:
for name, cons_searcher in cons_methods:
    
    # run search
    best_cons = cons_searcher.run(x_val, true_val)
    
    # make val and test predictions
    pred_val = cons_searcher._consensus_predict(x_val[best_cons])
    pred_test = cons_searcher._consensus_predict(x_test[best_cons])
    
    # write prediction accuracy metric
    df_val[name] = pred_val
    df_test[name] = pred_test

### 4. Summurize results

In [15]:
res = pd.DataFrame()
for model in df_val.columns[2:]:
    res.loc[model, "R2"] = r2_score(df_val["Y_TRUE"], df_val[model])

In [19]:
res.sort_values(by="R2", ascending=False)

,R2
Genetic,0.470755
Systematic,0.447144
Random,0.437362
BasicML|desc2D|MLPRegressor,0.392734
BasicML|desc2D|SVR,0.391462
...,...
BasicML|desc2D|DecisionTreeRegressor,-0.584921
BasicML|fcfp-count|DecisionTreeRegressor,-0.608773
BasicML|rdkit|RidgeRegression,-0.622639
BasicML|ecfp-count|DecisionTreeRegressor,-0.700100


In [32]:
y_pred = df_test["Genetic"].to_list()
results = benchmark.evaluate(y_pred)
results

{
  "benchmark_artifact_id": "polaris/adme-fang-solu-1",
  "benchmark_name": null,
  "benchmark_owner": null,
  "results": [
    {
      "test_set": "test",
      "target_label": "LOG_SOLUBILITY",
      "scores": {
        "explained_var": 0.35711736389720994,
        "pearsonr": 0.6004002542933552,
        "mean_squared_error": 0.3491398448007377,
        "r2": 0.3560427891695943,
        "mean_absolute_error": 0.4238736241606715,
        "spearmanr": 0.48347265962427716
      }
    }
  ],
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "0.13.0",
  "slug": null,
  "github_url": null,
  "paper_url": null,
  "contributors": [],
  "artifact_id": null,
  "urn": null,
  "version": 1
}

In [33]:
from polaris.hub.client import PolarisHubClient
with PolarisHubClient() as client:
    client.login()

[2025-10-06 07:45:35] INFO     Your browser has been opened to visit:                        ]8;id=553307;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/external_client.py\external_client.py]8;;\:]8;id=114754;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/external_client.py#149\149]8;;\
                               https://clerk.polarishub.io/oauth/authorize?response_type=cod                       
                               e&client_id=agQP2xVM6JqMHvGc&redirect_uri=https%3A%2F%2Fpolar                       
                               ishub.io%2Foauth2%2Fcallback&scope=profile+email&state=ZgvaW1                       
                               gA37U0xnGG2IhdaZ5WeAvCLy&code_challenge=VmLW-DCU83focOGYapvV5                       
                               MQefcOGEUx-5q90pTWMKrM&code_challenge_method=S256                                   
                                                                                                                   

Please enter the authorization token:  OGMZZGMWMTITYJQ3MI0ZOWJMLWEZNJGTYJY0YWI5ZDGXN2RK


[2025-10-06 07:46:02] INFO     Successfully authenticated to the Polaris Hub as              ]8;id=647632;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/external_client.py\external_client.py]8;;\:]8;id=937595;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/external_client.py#160\160]8;;\
                               `dvzankov@gmail.com`!                                                               

                      INFO     You are successfully logged in to the Polaris Hub.                     ]8;id=727505;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/client.py\client.py]8;;\:]8;id=64351;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/client.py#245\245]8;;\

In [34]:
results.name = "QSARcons"
results.upload_to_hub(owner="dzankov")

[07:46:09] Your result has been successfully uploaded to the Hub. View it here:                       ]8;id=912452;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/client.py\client.py]8;;\:]8;id=417758;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/hub/client.py#527\527]8;;\
           https://polarishub.io/benchmarks/polaris/adme-fang-solu-1/BRnT0vWtiWMe1z9pXD8s2                         

            Success: Uploading results                                                                ]8;id=877295;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=36455;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\